## Import Libraries

In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

## Import Data

In [2]:
df = pd.read_csv('./100_Unique_QA_Dataset.csv')

df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [3]:
df.shape

(90, 2)

## Data Processing

### Tokenize

In [4]:
def tokenize(text):
  text = text.lower()
  text = text.replace("?", "")
  text = text.replace("'", "")
  return text.split()

In [5]:
tokenize(df.iloc[2, 0])

['who', 'wrote', 'to', 'kill', 'a', 'mockingbird']

### Vocab

In [6]:
vocab = {'UNK': 0}                # unknown words

In [7]:
def build_vocab(row):
  tokenized_question = tokenize(row['question'])
  tokenized_answer = tokenize(row['answer'])

  merged_token = tokenized_question + tokenized_answer

  for token in merged_token:
    if token not in vocab:
      vocab[token] = len(vocab)

In [8]:
df.apply(build_vocab, axis=1)

0     None
1     None
2     None
3     None
4     None
      ... 
85    None
86    None
87    None
88    None
89    None
Length: 90, dtype: object

In [9]:
print(f'vocab size: {len(vocab)}')
vocab

vocab size: 324


{'UNK': 0,
 'what': 1,
 'is': 2,
 'the': 3,
 'capital': 4,
 'of': 5,
 'france': 6,
 'paris': 7,
 'germany': 8,
 'berlin': 9,
 'who': 10,
 'wrote': 11,
 'to': 12,
 'kill': 13,
 'a': 14,
 'mockingbird': 15,
 'harper-lee': 16,
 'largest': 17,
 'planet': 18,
 'in': 19,
 'our': 20,
 'solar': 21,
 'system': 22,
 'jupiter': 23,
 'boiling': 24,
 'point': 25,
 'water': 26,
 'celsius': 27,
 '100': 28,
 'painted': 29,
 'mona': 30,
 'lisa': 31,
 'leonardo-da-vinci': 32,
 'square': 33,
 'root': 34,
 '64': 35,
 '8': 36,
 'chemical': 37,
 'symbol': 38,
 'for': 39,
 'gold': 40,
 'au': 41,
 'which': 42,
 'year': 43,
 'did': 44,
 'world': 45,
 'war': 46,
 'ii': 47,
 'end': 48,
 '1945': 49,
 'longest': 50,
 'river': 51,
 'nile': 52,
 'japan': 53,
 'tokyo': 54,
 'developed': 55,
 'theory': 56,
 'relativity': 57,
 'albert-einstein': 58,
 'freezing': 59,
 'fahrenheit': 60,
 '32': 61,
 'known': 62,
 'as': 63,
 'red': 64,
 'mars': 65,
 'author': 66,
 '1984': 67,
 'george-orwell': 68,
 'currency': 69,
 'united

### Convert words to numerical indices

In [10]:
def text_to_indices(text, vocab):
  indexed_text = []

  for token in tokenize(text):
    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['UNK'])
  
  return indexed_text

In [11]:
text_to_indices('Who are you?', vocab)

[10, 81, 0]

## DataLoader

In [12]:
class QADataset(Dataset):

  def __init__(self, df, vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]
  
  def __getitem__(self, idx):
    numerical_qs = text_to_indices(self.df.iloc[idx]['question'], self.vocab)
    numerical_ans = text_to_indices(self.df.iloc[idx]['answer'], self.vocab)
    return torch.tensor(numerical_qs), torch.tensor(numerical_ans)

In [13]:
dataset = QADataset(df, vocab)
dataset[10]

(tensor([ 1,  2,  3,  4,  5, 53]), tensor([54]))

In [14]:
dataLoader = DataLoader(dataset, batch_size=1, shuffle=True)

## Model Definition

In [15]:
class SimpleRNN(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    # Cannot use nn.Sequential because RNN returns both output sequence and hidden state and we need to handle them explicitly for the next layer or loss calculation
    self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
    self.rnn = nn.RNN(50, 64, batch_first=True)
    self.fc = nn.Linear(64, vocab_size)

  def forward(self, qs):
    embedded_qs = self.embedding(qs)
    hidden, final = self.rnn(embedded_qs)
    output = self.fc(final.squeeze(0))
    return output

In [16]:
model = SimpleRNN(len(vocab))

## Training Pipeline

In [17]:
learning_rate = 0.001
epochs = 20

criterian = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [18]:
def training(epochs, model, loader):
  model.train()
  for epoch in range(epochs):
    total_loss =0
    for question, answer in loader:
      output = model(question)
      loss = criterian(output, answer[0])
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      total_loss += loss
    print(f'Epoch: {epoch+1} , Loss: {total_loss}')

In [19]:
training(epochs, model, dataLoader)

Epoch: 1 , Loss: 526.1122436523438
Epoch: 2 , Loss: 457.6596984863281
Epoch: 3 , Loss: 378.3297119140625
Epoch: 4 , Loss: 315.1127014160156
Epoch: 5 , Loss: 263.1197204589844
Epoch: 6 , Loss: 215.76986694335938
Epoch: 7 , Loss: 170.84909057617188
Epoch: 8 , Loss: 132.55682373046875
Epoch: 9 , Loss: 101.74592590332031
Epoch: 10 , Loss: 77.45215606689453
Epoch: 11 , Loss: 59.806549072265625
Epoch: 12 , Loss: 46.690677642822266
Epoch: 13 , Loss: 37.24738693237305
Epoch: 14 , Loss: 30.244945526123047
Epoch: 15 , Loss: 24.88571548461914
Epoch: 16 , Loss: 21.04729461669922
Epoch: 17 , Loss: 17.672645568847656
Epoch: 18 , Loss: 15.215731620788574
Epoch: 19 , Loss: 13.043421745300293
Epoch: 20 , Loss: 11.179683685302734


## Testing

In [20]:
def predict(model, qs, threshold=0.5):
  numerical_qs = text_to_indices(qs, vocab)
  qs_tensor = torch.tensor(numerical_qs).unsqueeze(0)
  output = model(qs_tensor)                            # output as 324 logits
  probs = nn.functional.softmax(output, dim=1)         # convert logits to probs
  value, idx = torch.max(probs, dim=1)

  if(value >= threshold and idx != 0):
    print(list(vocab.keys())[idx])
  else:
    print('Sorry! I do not know the answer to this question')

In [21]:
qs = 'capital of france'

predict(model, qs)

paris


## Summary

In [22]:
from torchinfo import summary

summary(model, input_size=(1, 6), dtypes=[torch.long])    # input size parameter

Layer (type:depth-idx)                   Output Shape              Param #
SimpleRNN                                [1, 324]                  --
├─Embedding: 1-1                         [1, 6, 50]                16,200
├─RNN: 1-2                               [1, 6, 64]                7,424
├─Linear: 1-3                            [1, 324]                  21,060
Total params: 44,684
Trainable params: 44,684
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.08
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.18
Estimated Total Size (MB): 0.19